## Data Description

You are provided with 25,010 poker hands in train.csv and 1,000,000 in test.csv. Each hand consists of five cards with a given suit and rank, drawn from a standard deck of 52. Suits and ranks are represented as ordinal categories:

```
S1 “Suit of card #1”
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}
C1 “Rank of card #1”
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

...

S5 “Suit of card #5”
C5 “Rank of card #5”
```

Each row in the training set has the accompanying class label for the poker hand it comprises. The hands are omitted from the test set and must be predicted by participants. Hands are classified into the following ordinal categories:


```
0: Nothing in hand; not a recognized poker hand 
1: One pair; one pair of equal ranks within five cards
2: Two pairs; two pairs of equal ranks within five cards
3: Three of a kind; three equal ranks within five cards
4: Straight; five cards, sequentially ranked with no gaps
5: Flush; five cards with the same suit
6: Full house; pair + different rank three of a kind
7: Four of a kind; four equal ranks within five cards
8: Straight flush; straight + flush
9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush
```
Note that the Straight flush and Royal flush hands are not representative of
the true domain because they have been over-sampled. The straight flush
is 14.43 times more likely to occur in the training set, while the royal flush is 129.82 times more likely.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from functools import reduce
import random

## Utils

#### Test function

In [2]:
def calc_hand_label(hand):    
    def f(hand):
        suits_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,0], {})
        ranks_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,1], {})
        
        if len(ranks_hist.values()) < 5:
            if len(ranks_hist.values()) == 2:
                if max(list(ranks_hist.values())) == 4:
                    return 7
    #                 print('7: Four of a kind; four equal ranks within five cards')
                else:
                    return 6
    #                 print('6 Full house; pair + different rank three of a kind')
            elif len(ranks_hist.values()) == 3:
                if max(list(ranks_hist.values())) == 3:
                    return 3
    #                 print('3: Three of a kind; three equal ranks within five cards')
                else:
                    return 2
    #                 print('2: Two pairs; two pairs of equal ranks within five cards')
            else:
                return 1
    #             print('1: One pair; one pair of equal ranks within five cards')
        else:
            if len(suits_hist.values()) == 1:
                if max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                    if max(list(ranks_hist.keys())) == 13:
                        return 9
    #                     print('9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush')
                    else:
                        return 8
    #                     print('8: Straight flush; straight + flush')
                else:
                    return 5
    #                 print('5: Flush; five cards with the same suit')
            elif max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                return 4    
    #             print('4: Straight; five cards, sequentially ranked with no gaps')
            else:
                return 0
                print('0: Nothing in hand; not a recognized poker hand ')
                
    hand = np.array(hand).reshape(5,2)
    res1 = f(hand)
    
    hand[:,1] = list(map(lambda x: 13 if (x == 1) else x-1, hand[:,1]))
    res2 = f(hand)
    
    if res1 == 9 and res2 == 8:
        return res2
    else:
        return max(res1, res2)

#### Encode cards

In [3]:
def card_encode(s, c):
    cards=["A","2","3","4","5","6","7","8","9","10","J","Q","K"]
    suits="♥♦♣♠"
    return cards[c - 1] + suits[s - 1]

def hand_encode(hand):
    return ', '.join(map(lambda x: card_encode(hand[x*2], hand[x*2+1]),range(5)))

print(hand_encode([4,9,2,1,2,2,4,7,2,8]))
print(hand_encode([2,9,2,4,3,6,1,9,4,9]))

9♠, A♦, 2♦, 7♠, 8♦
9♦, 4♦, 6♣, 9♥, 9♠


## Train data

In [4]:
def shuffle_suits_df(data):
    for index, row in data.iterrows():
        rank = -1
    
        row['S1'] = random.randint(1,4)
        row['S2'] = random.randint(1,4)
        row['S3'] = random.randint(1,4)
        row['S4'] = random.randint(1,4)
        row['S5'] = random.randint(1,4)
        rank = calc_hand_label(row[['S1', 'C1','S2', 'C2','S3', 'C3','S4', 'C4','S5', 'C5']])
    return data

In [5]:
suits = ['S1','S2','S3','S4','S5']
cards = ['C1','C2','C3','C4','C5']
distances = ['D1', 'D2', 'D3', 'D4', 'D5']

In [6]:
train_dataset = pd.read_csv('train.csv', delimiter=',')

In [7]:
straights = train_dataset[train_dataset['hand'] == 4]
train_dataset = train_dataset.append(shuffle_suits_df(straights),ignore_index=True).append(shuffle_suits_df(straights),ignore_index=True).append(shuffle_suits_df(straights),ignore_index=True).append(shuffle_suits_df(straights),ignore_index=True).append(shuffle_suits_df(straights),ignore_index=True).append(shuffle_suits_df(straights),ignore_index=True)

In [8]:
train_dataset.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,hand
0,4,9,2,1,2,2,4,7,2,8,0
1,1,4,3,6,1,12,3,11,2,7,0
2,1,11,4,1,3,7,4,11,2,1,2
3,2,9,2,4,3,6,1,9,4,9,3
4,1,8,2,4,2,11,2,2,2,1,0


In [9]:
data = train_dataset.drop('hand', axis=1)
train_labels = train_dataset['hand']

In [10]:
labels_vect = tf.keras.utils.to_categorical(train_labels)

#### Adding distance features

In [11]:
def get_distances(hand):
    distances_list = []
    for i, card in enumerate(hand):
        if (i < len(hand) - 1):
            distances_list.append(hand[i+1] - card)
        else:
            distances_list.append(13 - card + hand[0])
    return pd.Series(distances_list)

def with_distances(data):
    sorted = data[cards]
    sorted.values.sort()

    distances_df = sorted.apply(get_distances, axis=1)
    distances_df.columns = distances
    
    data_with_distances = pd.concat([data, distances_df], axis=1)
    return data_with_distances

In [12]:
data_with_distances = with_distances(data)
data_with_distances.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,D1,D2,D3,D4,D5
0,4,9,2,1,2,2,4,7,2,8,1,5,1,1,5
1,1,4,3,6,1,12,3,11,2,7,2,1,4,1,5
2,1,11,4,1,3,7,4,11,2,1,0,6,4,0,3
3,2,9,2,4,3,6,1,9,4,9,2,3,0,0,8
4,1,8,2,4,2,11,2,2,2,1,1,2,4,3,3


#### One hot encoding

In [13]:
def one_hot_encoded(data, columns, encoder, mode='transform'):
    encoded = None
    if (mode == 'fit'):
        encoded = encoder.fit_transform(data[columns])
    elif (mode == 'transform'):
        encoded = encoder.transform(data[columns])
    return pd.concat([data.drop(columns, axis=1), pd.DataFrame(encoded.toarray())], axis=1)

In [14]:
encoder = OneHotEncoder()
data_encoded = one_hot_encoded(data_with_distances, suits, encoder, 'fit')
print(data_encoded.shape)

data_encoded.head()

(25568, 30)


,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,...,10,11,12,13,14,15,16,17,18,19
0,9,1,2,7,8,1,5,1,1,5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,4,6,12,11,7,2,1,4,1,5,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,11,1,7,11,1,0,6,4,0,3,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,9,4,6,9,9,2,3,0,0,8,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,8,4,11,2,1,1,2,4,3,3,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Normalization

In [15]:
def normalized(data, columns, mean, std):
    data_normalized = (data[columns] - mean) / std
    to_return = pd.concat([data_normalized, data.drop(columns, axis=1)], axis=1)
    return to_return

In [16]:
mean_c = data_encoded[cards].mean().mean()
std_c = pd.Series(data_encoded[cards].values.flatten()).std()
mean_d = data_encoded[distances].mean().mean()
std_d = pd.Series(data_encoded[distances].values.flatten()).std()

data_normalized = normalized(data_encoded, cards, mean_c, std_c)
data_normalized = normalized(data_normalized, distances, mean_d, std_d)
data_normalized.head()

,D1,D2,D3,D4,D5,C1,C2,C3,C4,C5,...,10,11,12,13,14,15,16,17,18,19
0,-0.764302,1.146452,-0.764302,-0.764302,1.146452,0.534549,-1.603254,-1.336028,0.000098,0.267324,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,-0.286613,-0.764302,0.668764,-0.764302,1.146452,-0.801578,-0.267127,1.336225,1.069000,0.000098,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,-1.241990,1.624141,0.668764,-1.241990,0.191075,1.069000,-1.603254,0.000098,1.069000,-1.603254,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,-0.286613,0.191075,-1.241990,-1.241990,2.579518,0.534549,-0.801578,-0.267127,0.534549,0.534549,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.764302,-0.286613,0.668764,0.191075,0.191075,0.267324,-0.801578,1.069000,-1.336028,-1.603254,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [17]:
data_normalized.shape

(25568, 30)

## Neural Network

In [23]:
from time import time
from keras.callbacks import TensorBoard

model = keras.Sequential()
#input
model.add(keras.layers.Dense(120, activation='relu', input_shape=(30,)))
model.add(keras.layers.Dropout(0.2))
#hidden
model.add(keras.layers.Dense(240, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(120, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(60, activation='relu'))
model.add(keras.layers.Dropout(0.2))
# Add a softmax layer with 10 output units:
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(np.array(data_normalized), np.array(labels_vect), epochs=500, batch_size=512)

Epoch 1/500
25568/25568 [==============================]25568/25568 [==============================] - 3s 99us/step - loss: 1.4378 - acc: 0.4181

Epoch 2/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 37us/step - loss: 1.0998 - acc: 0.5095

Epoch 3/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 37us/step - loss: 1.0293 - acc: 0.5458

Epoch 4/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 39us/step - loss: 0.9391 - acc: 0.6033

Epoch 5/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 39us/step - loss: 0.8501 - acc: 0.6454

Epoch 6/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 39us/step - loss: 0.7850 - acc: 0.6800

Epoch 7/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 40us/step - loss: 

25568/25568 [==============================]25568/25568 [==============================] - 1s 40us/step - loss: 0.0161 - acc: 0.9957

Epoch 57/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 40us/step - loss: 0.0143 - acc: 0.9958

Epoch 58/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 38us/step - loss: 0.0140 - acc: 0.9961

Epoch 59/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 39us/step - loss: 0.0134 - acc: 0.9963

Epoch 60/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 38us/step - loss: 0.0129 - acc: 0.9964

Epoch 61/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 40us/step - loss: 0.0133 - acc: 0.9964

Epoch 62/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 38us/step - loss: 0.0117

25568/25568 [==============================]25568/25568 [==============================] - 1s 35us/step - loss: 0.0037 - acc: 0.9988

Epoch 111/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 36us/step - loss: 0.0040 - acc: 0.9986

Epoch 112/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 36us/step - loss: 0.0038 - acc: 0.9989

Epoch 113/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 43us/step - loss: 0.0025 - acc: 0.9992

Epoch 114/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 37us/step - loss: 0.0026 - acc: 0.9992

Epoch 115/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 38us/step - loss: 0.0027 - acc: 0.9993

Epoch 116/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 38us/step - loss: 

25568/25568 [==============================]25568/25568 [==============================] - 1s 33us/step - loss: 0.0027 - acc: 0.9991

Epoch 166/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 32us/step - loss: 0.0015 - acc: 0.9996

Epoch 167/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 33us/step - loss: 0.0020 - acc: 0.9993

Epoch 168/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 33us/step - loss: 0.0025 - acc: 0.9993

Epoch 169/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 34us/step - loss: 0.0017 - acc: 0.9995

Epoch 170/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 32us/step - loss: 0.0015 - acc: 0.9997

Epoch 171/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 33us/step - loss: 

25568/25568 [==============================]25568/25568 [==============================] - 1s 38us/step - loss: 0.0021 - acc: 0.9994

Epoch 221/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 45us/step - loss: 0.0017 - acc: 0.9995

Epoch 222/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 36us/step - loss: 0.0018 - acc: 0.9997

Epoch 223/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 37us/step - loss: 0.0022 - acc: 0.9995

Epoch 224/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 38us/step - loss: 0.0025 - acc: 0.9995

Epoch 225/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 36us/step - loss: 0.0014 - acc: 0.9996

Epoch 226/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 33us/step - loss: 

25568/25568 [==============================]25568/25568 [==============================] - 1s 39us/step - loss: 0.0011 - acc: 0.9998

Epoch 275/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 37us/step - loss: 6.2082e-04 - acc: 0.9998

Epoch 276/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 37us/step - loss: 0.0012 - acc: 0.9995

Epoch 277/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 35us/step - loss: 0.0014 - acc: 0.9997

Epoch 278/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 36us/step - loss: 8.9534e-04 - acc: 0.9997

Epoch 279/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 36us/step - loss: 4.6222e-04 - acc: 0.9999

Epoch 280/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 35us/s

25568/25568 [==============================]25568/25568 [==============================] - 1s 38us/step - loss: 0.0013 - acc: 0.9997

Epoch 329/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 39us/step - loss: 8.9453e-04 - acc: 0.9998

Epoch 330/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 39us/step - loss: 8.8086e-04 - acc: 0.9998

Epoch 331/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 39us/step - loss: 5.6974e-04 - acc: 0.9999

Epoch 332/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 42us/step - loss: 6.0201e-04 - acc: 0.9998

Epoch 333/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 38us/step - loss: 9.3014e-04 - acc: 0.9996

Epoch 334/500
25568/25568 [==============================]25568/25568 [==============================] - 1

25568/25568 [==============================]25568/25568 [==============================] - 1s 39us/step - loss: 6.3076e-04 - acc: 0.9997

Epoch 383/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 39us/step - loss: 0.0015 - acc: 0.9996

Epoch 384/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 40us/step - loss: 6.6296e-04 - acc: 0.9999

Epoch 385/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 41us/step - loss: 5.2803e-04 - acc: 0.9998

Epoch 386/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 38us/step - loss: 7.0152e-04 - acc: 0.9998

Epoch 387/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 40us/step - loss: 7.5393e-04 - acc: 0.9998

Epoch 388/500
25568/25568 [==============================]25568/25568 [==============================] - 1

25568/25568 [==============================]25568/25568 [==============================] - 1s 39us/step - loss: 8.4493e-04 - acc: 0.9997

Epoch 437/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 38us/step - loss: 3.5487e-04 - acc: 0.9999

Epoch 438/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 37us/step - loss: 2.5352e-04 - acc: 1.0000

Epoch 439/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 38us/step - loss: 3.1803e-04 - acc: 1.0000

Epoch 440/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 46us/step - loss: 7.9359e-04 - acc: 0.9998

Epoch 441/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 40us/step - loss: 6.8053e-04 - acc: 0.9999

Epoch 442/500
25568/25568 [==============================]25568/25568 [==============================]

25568/25568 [==============================]25568/25568 [==============================] - 1s 38us/step - loss: 8.6799e-04 - acc: 0.9998

Epoch 491/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 38us/step - loss: 8.2773e-04 - acc: 0.9997

Epoch 492/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 39us/step - loss: 4.1859e-04 - acc: 0.9998

Epoch 493/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 36us/step - loss: 1.5259e-04 - acc: 1.0000

Epoch 494/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 38us/step - loss: 0.0010 - acc: 0.9996

Epoch 495/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 40us/step - loss: 0.0016 - acc: 0.9994

Epoch 496/500
25568/25568 [==============================]25568/25568 [==============================] - 1s 52

In [28]:
model.save('./model-35')

## Test data

In [19]:
test_dataset = pd.read_csv('test.csv', delimiter=',')
test_data = test_dataset.drop('id', axis=1)

In [20]:
test_labels = test_data.apply(calc_hand_label, axis=1)

In [21]:
test_data_with_distances = with_distances(test_data)
test_data_encoded = one_hot_encoded(test_data_with_distances, suits, encoder)
test_data_normalized = normalized(test_data_encoded, cards, mean_c, std_c)
test_data_normalized = normalized(test_data_normalized, distances, mean_d, std_d)

In [22]:
test_data_normalized.shape

(1000000, 30)

## Predictions

In [24]:
test_data_sample_predicted_10 = model.predict(np.array(test_data_normalized))
test_data_sample_predicted = np.array(list(map(np.argmax, test_data_sample_predicted_10)))

In [25]:
predictions_data = pd.concat([
    test_dataset,
    pd.DataFrame({'predictions': test_data_sample_predicted}),
    pd.DataFrame({'labels': test_labels}),
], axis=1)

In [26]:
wrong_answers = predictions_data[predictions_data['predictions'] != predictions_data['labels']]
wrong_answers.shape

(35, 13)

In [27]:
wrong_answers['labels'].value_counts()

0    23
5    11
8     1
Name: labels, dtype: int64

In [30]:
wrong_answers

,id,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,predictions,labels
37205,37206,2,10,2,7,2,11,2,13,2,8,0,5
48013,48014,2,6,2,8,2,12,2,9,2,11,0,5
65550,65551,4,2,4,13,4,1,4,3,4,10,0,5
100072,100073,4,10,2,11,3,13,2,1,1,9,4,0
118487,118488,4,9,2,10,3,13,2,1,3,11,4,0
118862,118863,3,12,2,11,2,10,2,2,2,13,4,0
132149,132150,4,11,2,9,1,12,2,1,1,10,4,0
133716,133717,1,9,2,10,4,12,1,1,2,11,4,0
139746,139747,4,1,4,11,4,13,4,9,3,10,4,0
196807,196808,1,9,2,13,4,11,4,1,4,10,4,0


In [32]:
wrong_answers.drop(['id', 'predictions', 'labels'], axis=1).apply(lambda h: hand_encode(np.array(h)), axis=1)

37205     10♦, 7♦, J♦, K♦, 8♦
48013      6♦, 8♦, Q♦, 9♦, J♦
65550     2♠, K♠, A♠, 3♠, 10♠
100072    10♠, J♦, K♣, A♦, 9♥
118487    9♠, 10♦, K♣, A♦, J♣
118862    Q♣, J♦, 10♦, 2♦, K♦
132149    J♠, 9♦, Q♥, A♦, 10♥
133716    9♥, 10♦, Q♠, A♥, J♦
139746    A♠, J♠, K♠, 9♠, 10♣
196807    9♥, K♦, J♠, A♠, 10♠
263736    10♣, 9♣, K♠, A♠, J♠
292845    Q♦, J♦, K♦, 9♦, 10♦
293666    9♠, K♦, 10♠, A♥, J♥
304204     4♣, K♣, A♣, Q♣, 2♣
348642    A♥, 9♣, K♠, J♠, 10♦
398465    10♦, Q♣, K♠, 2♠, J♠
401356    J♦, 10♦, K♦, A♦, 9♥
498074    Q♠, 10♦, J♠, A♦, 9♥
589450    J♥, 10♣, K♣, 2♣, Q♣
601357    Q♠, K♦, 10♠, 2♦, J♦
601654     6♦, 7♦, K♦, 8♦, J♦
612116    J♥, Q♥, K♣, 2♦, 10♣
636637     2♠, J♠, 5♠, A♠, K♠
636898    9♠, 10♦, J♠, A♥, K♣
646682    K♥, J♦, Q♥, 2♠, 10♣
663173     A♠, Q♠, 7♠, 3♠, K♠
703959     7♦, K♦, J♦, 9♦, Q♦
706031    9♥, J♦, Q♦, A♦, 10♣
809892     Q♦, J♦, K♦, 9♦, 8♦
818397    10♥, K♦, J♠, A♥, 9♠
833098    J♣, 9♣, K♠, A♠, 10♦
866415    J♠, Q♦, K♠, 2♠, 10♣
893283    J♦, 7♦, Q♦, 9♦, 10♦
915377    